In [1]:
import requests
import json
import re
import pandas as pd
import datetime
from bs4 import BeautifulSoup as soup

In [2]:
def gethtml(url2):
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.75 Safari/537.36"
    }
    #獲取資料
    req = requests.get(url=url2, headers=headers)
    req.encoding = "utf-8"
    return req.text

In [3]:
def AllSubHtm(): ##取得QS各分領域排名的網頁##
    urlRankN="https://www.topuniversities.com/subject-rankings/2017"##改年分##
    temp1=gethtml(urlRankN)
    a=[]
    sub_htm=soup(temp1,'html.parser')
    for item in sub_htm.find_all('a',href=re.compile("^/university-rankings/university-subject-rankings/2017/")): ##改年分##
        a.append('https://www.topuniversities.com/'+item.get('href'))
    return a

In [4]:
def getSubDatahtm(url): ##在分領域排名的網頁內取得資料檔的連結##
    temp=gethtml(url)
    rank_htm=soup(temp,'html.parser')
    data_htm=rank_htm.find('script',type='application/json')
    str_data_htm=str(data_htm)
    start=str_data_htm.find('rank_indicators_url')
    end=str_data_htm.find('compare_url')
    result=str_data_htm[int(start)+22:int(end)-3]                     
    return  result.replace('\/','/')

In [5]:
def getSubData(Sub_Rank_url):
    res_url=getSubDatahtm(Sub_Rank_url)
    result=gethtml(res_url)
    JResult=json.loads(result)
    SubData=JResult['data']
    return SubData

In [6]:
def iferror(success, failure, *exceptions): ##可以拿來解決錯誤問題，但好像要配合lambda##
    try:
        return success()
    except exceptions or Exception:
        return failure

In [11]:
##單檔版##
##主程式區##
list=AllSubHtm()
y=[]
for j in range(0,len(list)):
    subdata=getSubData(list[j])
    x=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]    
    for i in range (0,len(subdata)):
        x[0]=subdata[i]['nid']                                   ##校id##
        unithtm=soup(subdata[i]['uni'],'html.parser')            ##校名##
        x[1]=unithtm.find('a').text
        x[2]=subdata[i]['region']                                ##區域(州)## 
        x[3]=subdata[i]['location']                              ##地區(國家)## 
        x[4]=subdata[i]['city']                                  ##城市##
        x[5]=subdata[i]['overall_rank']                          ##總排名##
        overallhtm=soup(subdata[i]['overall'],'html.parser')     ##總分##
        x[6]=overallhtm.find('div').text                
        x[7]=iferror(lambda: subdata[i]['rank_76'],"NAN")                               ##學術聲望排名##
        ind_76htm=iferror(lambda: soup(subdata[i]['ind_76'],'html.parser'),"NAN")       ##學術聲望分數##
        x[8]=iferror(lambda: ind_76htm.find('div',class_='td-wrap-in').text,"NAN")
        x[9]=iferror(lambda: subdata[i]['rank_77'],"NAN")                               ##雇主聲望排名##
        ind_77htm=iferror(lambda: soup(subdata[i]['ind_77'],'html.parser'),"NAN")       ##雇主聲望分數##
        x[10]=iferror(lambda: ind_77htm.find('div').text,"NAN")                         
        x[11]=iferror(lambda: subdata[i]['rank_69'],"NAN")                              ##H指數排名##
        ind_69htm=iferror(lambda: soup(subdata[i]['ind_69'],'html.parser'),"NAN")       ##H指數分數##
        x[12]=iferror(lambda: ind_69htm.find('div').text,"NAN")
        x[13]=iferror(lambda: subdata[i]['rank_70'],"NAN")                              ##篇均被引排名##
        ind_70htm=iferror(lambda: soup(subdata[i]['ind_70'],'html.parser'),"NAN")       ##篇軍被引分數##
        x[14]=iferror(lambda: ind_70htm.find('div').text,"NAN")
        x[15]=iferror(lambda: subdata[i]['stars'],"NAN")                                ##QS星註記##
        x[16]=iferror(lambda: str(list[j][86:]),"NAN")                                  ##領域註記##
        y.append(x)
        x=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    print (str(list[j][86:])+':'+str(datetime.datetime.now()))

df=pd.DataFrame(data=y[0::],columns=['nid','university','region','location','city','overall_rank','overall_score','acadamic_reputaion_rank','acadamic_reputaion_score','employer_reputaion_rank','employer_reputaion_score','Hindex_rank','Hindex_score','citaion_rank','citaion_score','QSstars','Subject'])
df.to_csv("QS_SubRank2017/QS2017subject_all.csv",index=0,encoding='utf_8_sig') ##位址與檔名都改年分##       
y.clear()
del df  
print('finished')



arts-humanities:2022-02-17 10:04:05.622981
archaeology:2022-02-17 10:04:08.688582
architecture:2022-02-17 10:04:14.488474
art-design:2022-02-17 10:04:17.714828
english-language-literature:2022-02-17 10:04:24.295335
history:2022-02-17 10:04:26.592649
linguistics:2022-02-17 10:04:29.299910
modern-languages:2022-02-17 10:04:30.883745
performing-arts:2022-02-17 10:04:33.885520
philosophy:2022-02-17 10:04:35.700056
theology-divinity-religious-studies:2022-02-17 10:04:37.929575
engineering-technology:2022-02-17 10:04:39.909180
engineering-chemical:2022-02-17 10:04:41.119156
engineering-civil-structural:2022-02-17 10:04:42.868154
computer-science-information-systems:2022-02-17 10:04:44.415474
engineering-electrical-electronic:2022-02-17 10:04:46.277061
engineering-mechanical:2022-02-17 10:04:49.051493
engineering-mineral-mining:2022-02-17 10:04:50.661382
life-sciences-medicine:2022-02-17 10:04:53.710432
agriculture-forestry:2022-02-17 10:04:55.436821
anatomy-physiology:2022-02-17 10:04:57.053

In [ ]:
##測試區分隔線##

In [10]:
##取得分領域排名網頁##
urlRankN="https://www.topuniversities.com/subject-rankings/2017"
temp1=gethtml(urlRankN)
sub_htm=soup(temp1,'html.parser')
for item in sub_htm.find_all('a',href=re.compile("^/university-rankings/university-subject-rankings/2020/")):
    print('https://www.topuniversities.com/'+item.get('href'))
xxx=AllSubHtm()
print (xxx[3][86:])

art-design


In [116]:
##全爬法_備用##
url2021="https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/3518694_indicators.txt?1618844814"
##url2020="https:\/\/www.topuniversities.com\/sites\/default\/files\/qs-rankings-data\/en\/914824_indicators.txt?1618912359"##
##url2021arc="https://www.topuniversities.com//sites//default//files//qs-rankings-data//en//3515256_indicators.txt?1618912887"##
urls=getSubDatahtm('https://www.topuniversities.com//university-rankings/university-subject-rankings/2021/arts-humanities')
                   
result=gethtml(urls)
j_result=json.loads(result) ##結果以dic形式儲存，分為column與data兩大組##
##column收錄資料欄位名稱和一些其他屬性，每個串列存一個欄位名稱與其其他屬性##
##data為資料集，每個串列存一個學校，其內容以dic儲存，column裡的data就是dic的key##
a=j_result['columns']
header='nid'
for col_i in range (0,len(a)):
    header=header+'|'+str(a[col_i]['data'])
header=header+'\n'  ##建立csv格式的表頭##
line=''
b=j_result['data']
for data_i in range (0,len(b)):
    line=line+str(b[data_i]['nid'])
    for col_i in range (0,len(a)):
        line=line+'|'+str(b[data_i][a[col_i]['data']])
    line=line+'\n'
output=header+line
print (output)
        



nid|region|location|overall_rank|overall_rank_dis|uni|overall|stars|guide|recm|city|ind_69|rank_d_69|rank_69|ind_70|rank_d_70|rank_70|ind_76|rank_d_76|rank_76|ind_77|rank_d_77|rank_77
294654|Europe|United Kingdom|1|<div class="td-wrap"><div class="td-wrap-in">1</div></div>|<div class="td-wrap"><div class="td-wrap-in"><a href="/universities/university-oxford" class="uni-link">University of Oxford</a></div></div>|<div class="td-wrap"><div class="td-wrap-in">98.6</div></div>|||0--|Oxford|<div class="td-wrap"><div class="td-wrap-in">100</div></div>|<div class="td-wrap"><div class="td-wrap-in">1</div></div>|1|<div class="td-wrap"><div class="td-wrap-in">93.9</div></div>|<div class="td-wrap"><div class="td-wrap-in">11</div></div>|11|<div class="td-wrap"><div class="td-wrap-in">100</div></div>|<div class="td-wrap"><div class="td-wrap-in">1</div></div>|1|<div class="td-wrap"><div class="td-wrap-in">96.1</div></div>|<div class="td-wrap"><div class="td-wrap-in">2</div></div>|2
294270|North Ameri

In [12]:
##取得資料檔測試區##
url='https://www.topuniversities.com//university-rankings/university-subject-rankings/2017/art-design'
temp2=gethtml(url)
rank_htm=soup(temp2,'html.parser')
data_htm=rank_htm.find('script',type='application/json')
str_data_htm=str(data_htm)
start=str_data_htm.find('rank_indicators_url')
end=str_data_htm.find('compare_url')
print(str_data_htm[int(start)+22:int(end)-3])

print (getSubDatahtm(url))



https:\/\/www.topuniversities.com\/sites\/default\/files\/qs-rankings-data\/en\/335227_indicators.txt?1619686735
https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/335227_indicators.txt?1619686735


In [ ]:
x=getSubDatahtm('https://www.topuniversities.com//university-rankings/university-subject-rankings/2021/arts-humanities')
result=gethtml(x)
j_x=json.loads(result)
xdata=j_x['data']
print(xdata[0])



In [157]:
##主程式區##
subdata=getSubData('https://www.topuniversities.com//university-rankings/university-subject-rankings/2021/arts-humanities')
print (subdata[0])



unithtm=soup(subdata[0]['uni'],'html.parser')
print (unithtm.find('a').text)

overallhtm=soup(subdata[0]['overall'],'html.parser')
print (overallhtm.find('div').text)

ind_76htm=soup(subdata[0]['ind_76'],'html.parser')
print (ind_76htm.find('div',class_='td-wrap-in').text)

ind_77htm=soup(subdata[0]['ind_77'],'html.parser')
print (ind_77htm.find('div').text)

ind_69htm=soup(subdata[0]['ind_69'],'html.parser')
print (ind_69htm.find('div').text)

ind_70htm=soup(subdata[0]['ind_70'],'html.parser')
print (ind_70htm.find('div').text)


{'nid': '294654', 'region': 'Europe', 'location': 'United Kingdom', 'city': 'Oxford', 'overall_rank': '1', 'overall_rank_dis': '<div class="td-wrap"><div class="td-wrap-in">1</div></div>', 'uni': '<div class="td-wrap"><div class="td-wrap-in"><a href="/universities/university-oxford" class="uni-link">University of Oxford</a></div></div>', 'overall': '<div class="td-wrap"><div class="td-wrap-in">98.6</div></div>', 'stars': '', 'guide': '', 'recm': '0--', 'ind_76': '<div class="td-wrap"><div class="td-wrap-in">100</div></div>', 'rank_d_76': '<div class="td-wrap"><div class="td-wrap-in">1</div></div>', 'rank_76': '1', 'ind_77': '<div class="td-wrap"><div class="td-wrap-in">96.1</div></div>', 'rank_d_77': '<div class="td-wrap"><div class="td-wrap-in">2</div></div>', 'rank_77': '2', 'ind_69': '<div class="td-wrap"><div class="td-wrap-in">100</div></div>', 'rank_d_69': '<div class="td-wrap"><div class="td-wrap-in">1</div></div>', 'rank_69': '1', 'ind_70': '<div class="td-wrap"><div class="td-

In [7]:
##主程式區(一領域一檔版)##
list=AllSubHtm()
for j in  range(0,len(list)):
    print (str(list[j][86:]))
    subdata=getSubData(list[j])
    x=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    y=[]
    for i in range (0,len(subdata)):
        x[0]=subdata[i]['nid']                                   ##校id##
        unithtm=soup(subdata[i]['uni'],'html.parser')            ##校名##
        x[1]=unithtm.find('a').text
        x[2]=subdata[i]['region']                                ##區域(州)## 
        x[3]=subdata[i]['location']                              ##地區(國家)## 
        x[4]=subdata[i]['city']                                  ##城市##
        x[5]=subdata[i]['overall_rank']                          ##總排名##
        overallhtm=soup(subdata[i]['overall'],'html.parser')     ##總分##
        x[6]=overallhtm.find('div').text                
        x[7]=iferror(subdata[i]['rank_76'],"NAN")                               ##學術聲望排名##
        ind_76htm=iferror(lambda: soup(subdata[i]['ind_76'],'html.parser'),"NAN")       ##學術聲望分數##
        x[8]=iferror(lambda: ind_76htm.find('div',class_='td-wrap-in').text,"NAN")
        x[9]=iferror(lambda: subdata[i]['rank_77'],"NAN")                               ##雇主聲望排名##
        ind_77htm=iferror(lambda: soup(subdata[i]['ind_77'],'html.parser'),"NAN")       ##雇主聲望分數##
        x[10]=iferror(lambda: ind_77htm.find('div').text,"NAN")                         
        x[11]=iferror(lambda: subdata[i]['rank_69'],"NAN")                              ##H指數排名##
        ind_69htm=iferror(lambda: soup(subdata[i]['ind_69'],'html.parser'),"NAN")       ##H指數分數##
        x[12]=iferror(lambda: ind_69htm.find('div').text,"NAN")
        x[13]=iferror(lambda: subdata[i]['rank_70'],"NAN")                              ##篇均被引排名##
        ind_70htm=iferror(lambda: soup(subdata[i]['ind_70'],'html.parser'),"NAN")       ##篇軍被引分數##
        x[14]=iferror(lambda: ind_70htm.find('div').text,"NAN")
        x[15]=subdata[i]['stars']                                ##QS星註記##
        x[16]=str(list[j][86:])
        y.append(x)
        x=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    df=pd.DataFrame(data=y[0::],columns=['nid','university','region','location','city','overall_rank','overall_score','acadamic_reputaion_rank','acadamic_reputaion_score','employer_reputaion_rank','employer_reputaion_score','Hindex_rank','Hindex_score','citaion_rank','citaion_score','QSstars','Subject'])
    df.to_csv("QS_SubRank2017/"+str(list[j][86:])+"_2017.csv",index=0,encoding='‘utf_8_sig') ##位址與檔名都改年分##
    y.clear()
    del df
    print (str(datetime.datetime.now()))
print('finished')


arts-humanities
2021-05-04 14:49:51.406255
archaeology
2021-05-04 14:49:53.520381
architecture
2021-05-04 14:49:55.299610
art-design
2021-05-04 14:49:57.590023
english-language-literature
2021-05-04 14:50:00.092692
history
2021-05-04 14:50:02.710075
linguistics
2021-05-04 14:50:07.711906
modern-languages
2021-05-04 14:50:14.095247
performing-arts
2021-05-04 14:50:16.420371
philosophy
2021-05-04 14:50:18.372517
theology-divinity-religious-studies
2021-05-04 14:50:20.400745
engineering-technology
2021-05-04 14:50:22.882357
engineering-chemical
2021-05-04 14:50:25.100624
engineering-civil-structural
2021-05-04 14:50:27.185766
computer-science-information-systems
2021-05-04 14:50:28.492699
engineering-electrical-electronic
2021-05-04 14:50:30.991641
engineering-mechanical
2021-05-04 14:50:32.203911
engineering-mineral-mining
2021-05-04 14:50:34.816379
life-sciences-medicine
2021-05-04 14:50:37.185995
agriculture-forestry
2021-05-04 14:50:39.542039
anatomy-physiology
2021-05-04 14:50:41.640